In [8]:
import power_system_simulation as pss
import pandas as pd
from power_grid_model.utils import json_deserialize_from_file
import numpy as np
import graph_processing as gp


In [9]:
dataset = json_deserialize_from_file("input_network_data_assign3.json")

print(type(dataset))
print(pd.DataFrame(dataset.keys()))

# print(pd.DataFrame(dataset['line']))
# print(pd.DataFrame(dataset['node']))
# print(pd.DataFrame(dataset['transformer']))
# print(pd.DataFrame(dataset['source']))
# print(pd.DataFrame(dataset['sym_load']))

<class 'dict'>
             0
0         node
1         line
2  transformer
3       source
4     sym_load


In [10]:
reactive_power_data = pd.read_parquet('reactive_power_profile_assign3.parquet')

print(reactive_power_data)

Load ID                        12            13            14            15
Timestamp                                                                  
2025-01-01 00:00:00  -4125.774726  10138.692087   1128.989344   2737.227881
2025-01-01 00:15:00  16401.177195 -18890.906272 -15686.701911 -15440.509838
2025-01-01 00:30:00  14168.764135  10004.458792  12476.299329  18277.445982
2025-01-01 00:45:00  12099.434484  -1516.888466   9174.032490 -15571.787249
2025-01-01 01:00:00   5545.372267 -13336.327420  16457.603682    807.547193
...                           ...           ...           ...           ...
2025-01-10 22:45:00  20312.080701   7666.545676    793.539925  -3537.943985
2025-01-10 23:00:00 -15947.820470   -916.845584 -17261.491025   3196.493251
2025-01-10 23:15:00   8482.832761  15930.625066  -8487.686743  -4529.456669
2025-01-10 23:30:00 -15291.785929  14796.648255  12155.264848  11322.252550
2025-01-10 23:45:00 -18048.979231   2193.901032 -12565.357723 -15418.811806

[960 rows x

In [11]:
active_power_data = pd.read_parquet('active_power_profile_assign3.parquet')
print(type(active_power_data))

print(active_power_data)

<class 'pandas.core.frame.DataFrame'>
Load ID                        12            13            14            15
Timestamp                                                                  
2025-01-01 00:00:00  53845.936311  52868.809400  59437.379259  61193.999654
2025-01-01 00:15:00  58623.012703  66992.056154  57797.477123  48956.385676
2025-01-01 00:30:00  64800.016277  54713.600702  51293.354771  58092.153016
2025-01-01 00:45:00  61525.469129  59903.401304  49747.760990  62055.246675
2025-01-01 01:00:00  60289.133403  56883.894966  56301.650801  56226.510409
...                           ...           ...           ...           ...
2025-01-10 22:45:00  69484.715910  71348.849463  47979.322152  64352.546069
2025-01-10 23:00:00  61503.570916  60836.586259  76135.394978  59164.472679
2025-01-10 23:15:00  56045.155706  54275.886219  63557.959847  66834.711500
2025-01-10 23:30:00  73370.995338  59556.779716  57056.774548  62640.438159
2025-01-10 23:45:00  62018.984783  58840.504085  7

In [12]:

# declare new PowerSimModel object
test1 = pss.PowerSim(grid_data=dataset)

# PowerSim saves the model as PowerSimModel
# access to assignment 2 functions is now given as 
dataset = test1.PowerSimModel.batch_powerflow(active_power_profile=active_power_data, reactive_power_profile=reactive_power_data)
aggregate_test = test1.PowerSimModel.aggregate_voltage_table(active_power_profile=active_power_data, reactive_power_profile=reactive_power_data)

# PowerSim saves the model as PowerSimModel
# access to assignment 3 functions is now
print(test1.example_code())



# (active_power_profile=active_power_data, reactive_power_profile=reactive_power_data)

Who reads trek een bak
None


## N-1 testing stuff
- Firstly rewriting the input for assignment 1 function "alternative edges"
- Then running the alternative edges function
- For each alternative `line` to be connected, running the power flow calculation, with the original input type (?)
- Make a table, similar to the loading table of assignment 2 with each row being an alternative scenario, containing:
    - Alternative ID to be connected
    - Maximum loading
    - Line ID of the maximum
    - Timestamp of the maximum

In [20]:
dataset = json_deserialize_from_file("input_network_data_assign3.json")

test1 = pss.PowerSim(grid_data=dataset)

line_disabled = 16

# Now rewriting the output data to assignment 1 type of data:

edge_vertex_id_pairs = list(zip(dataset["line"]["from_node"], dataset["line"]["to_node"])) + list(zip(dataset["transformer"]["from_node"], dataset["transformer"]["to_node"]))
edge_enabled = []
for i in dataset["line"]["id"]:
    index = np.where(dataset["line"]["id"] == i)
    if dataset["line"][index]["from_status"] == 1 & dataset["line"][index]["to_status"] == 1:
        edge_enabled = edge_enabled + [True]
    else:
        edge_enabled = edge_enabled + [False]
if dataset["transformer"][0]["from_status"] == 1 & dataset["transformer"][0]["to_status"] == 1:
    edge_enabled = edge_enabled + [True]
else:
    edge_enabled = edge_enabled + [False]
source_vertex_id = dataset["source"]["node"][0]
edge_ids = list(dataset["line"]["id"]) + list(dataset["transformer"]["id"])
vertex_ids = dataset["node"]["id"]

# Find alternative edges

test2 = gp.GraphProcessor(
    vertex_ids=vertex_ids,
    edge_ids=edge_ids,
    edge_vertex_id_pairs=edge_vertex_id_pairs,
    edge_enabled=edge_enabled,
    source_vertex_id=source_vertex_id,
    )

alt_edges = test2.find_alternative_edges(line_disabled)

# Run Powerflow table and aggregate table

results = []

line_data = dataset["line"]


for alt_line_id in alt_edges:
    alt_line_index = None
    for i in range(len(line_data["id"])):
        if line_data["id"][i] == alt_line_id:
            alt_line_index = i
            break
    if alt_line_index is not None:
        line_data["to_status"][alt_line_index] = 1  
        output_data = test1.PowerSimModel.batch_powerflow(active_power_profile=active_power_data, reactive_power_profile=reactive_power_data)
        loading_table = test1.PowerSimModel.aggregate_loading_table(active_power_profile=active_power_data, reactive_power_profile=reactive_power_data)

        max_loading = loading_table["Max_Loading"].max()
        max_loading_id = loading_table["Max_Loading"].idxmax()
        max_loading_timestamp = loading_table.loc[max_loading_id, "Max_Loading_Timestamp"]

        results.append({
            "Alternative_Line_ID": alt_line_id,
            "Max_Loading": max_loading,
            "Max_Loading_ID": max_loading_id,
            "Max_Loading_Timestamp": max_loading_timestamp
        })
results_df = pd.DataFrame(results)
if results_df.empty:
    results_df = pd.DataFrame(columns=["Alternative_Line_ID", "Max_Loading", "Max_Loading_ID", "Max_Loading_Timestamp"])

results_df


,Alternative_Line_ID,Max_Loading,Max_Loading_ID,Max_Loading_Timestamp
0,24,0.001659,21,2025-01-07 10:30:00
